In [1]:


import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import datasets
import numpy as np
import os


2022-12-07 02:31:34.478623: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 02:31:34.552493: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-07 02:31:34.573165: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 02:31:34.957946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:



%%time
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, input_shape=[], dtype=tf.string, trainable=True)


2022-12-07 02:31:37.112178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:31:37.132283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:31:37.132400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 02:31:37.132766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

CPU times: user 5.78 s, sys: 504 ms, total: 6.29 s
Wall time: 6.36 s


In [ ]:
n = df.course_dept.nunique()
print(n)

In [3]:


def build_model(embed):
    
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        Dropout(0.2),
        Dense(190, activation='softmax')
    ])
    model.compile(Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model



In [4]:
model = build_model(embed)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               147354880 
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 190)               48830     
                                                                 
Total params: 147,535,038
Trainable params: 147,535,038
Non-trainable params: 0
_________________________________________________________________


In [7]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
df = pd.read_csv("https://waf.cs.illinois.edu/discovery/course-catalog.csv", )
df.head()
df2 = pd.DataFrame()
df2["text"] = df["Description"]
df2["label"] = preprocessing.LabelEncoder().fit_transform(df["Subject"])

x_train, x_test, y_train, y_test = train_test_split(df2["text"].values, df2["label"].values, test_size=0.2, random_state=42)
x_train = np.asarray(x_train).astype(np.str)

listMajors = df["Subject"].unique()

/tmp/ipykernel_243579/3747785483.py:12: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_train = np.asarray(x_train).astype(np.str)


In [8]:
print(len(x_train))
print(len(y_train))
print(listMajors)

10114
10114
['AAS' 'ABE' 'ACCY' 'ACE' 'ACES' 'ADV' 'AE' 'AFAS' 'AFRO' 'AFST' 'AGCM'
 'AGED' 'AHS' 'AIS' 'ALEC' 'ANSC' 'ANTH' 'ARAB' 'ARCH' 'ART' 'ARTD' 'ARTE'
 'ARTF' 'ARTH' 'ARTJ' 'ARTS' 'ASRM' 'ASST' 'ASTR' 'ATMS' 'BADM' 'BASQ'
 'BCOG' 'BCS' 'BDI' 'BIOC' 'BIOE' 'BIOP' 'BSE' 'BTW' 'BUS' 'CB' 'CDB'
 'CEE' 'CHBE' 'CHEM' 'CHIN' 'CHLH' 'CHP' 'CI' 'CIC' 'CLCV' 'CLE' 'CMN'
 'CPSC' 'CS' 'CSE' 'CW' 'CWL' 'CZCH' 'DANC' 'DTX' 'EALC' 'ECE' 'ECON'
 'EDPR' 'EDUC' 'EIL' 'ENG' 'ENGL' 'ENSU' 'ENVS' 'EPOL' 'EPS' 'EPSY' 'ERAM'
 'ESE' 'ESL' 'ETMA' 'EURO' 'FAA' 'FIN' 'FLTE' 'FR' 'FSHN' 'GC' 'GEOL'
 'GER' 'GGIS' 'GLBL' 'GMC' 'GRK' 'GRKM' 'GS' 'GSD' 'GWS' 'HDFS' 'HEBR'
 'HIST' 'HNDI' 'HORT' 'HRD' 'HT' 'HUM' 'IB' 'IE' 'IHLT' 'INFO' 'IS' 'ITAL'
 'JAPN' 'JOUR' 'JS' 'KIN' 'KOR' 'LA' 'LAS' 'LAST' 'LAT' 'LAW' 'LCTL'
 'LEAD' 'LER' 'LING' 'LLS' 'MACS' 'MATH' 'MBA' 'MCB' 'MDIA' 'MDVL' 'ME'
 'MFST' 'MICR' 'MILS' 'MIP' 'MSE' 'MUS' 'MUSC' 'MUSE' 'NE' 'NEUR' 'NPRE'
 'NRES' 'NS' 'NUTR' 'PATH' 'PERS' 'PHIL' 'PHYS' 'PLPA'

In [ ]:


model.load_weights('model.h5')

checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(x_train, 
          y_train, 
          batch_size=64, 
          validation_data=(x_test, y_test),
          callbacks=[checkpoint],
          epochs=15)

In [9]:


aux_model = tf.keras.Model(inputs=model.inputs,
                          outputs=model.outputs + [model.get_layer("keras_layer").output])

final_output, intermediate_layer_output = aux_model.predict(['I want to study computers'])

#print(final_output)
print(len(intermediate_layer_output[0]))

2022-12-07 02:32:08.310766: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 4s 4s/step
512


2022-12-07 02:32:09.073301: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [42]:
model.load_weights('model.h5')
test_pred = model.predict(['I want to study computers'])
#listMajors = sorted(os.listdir("/home/dylan/Desktop/batman/CS478-LinguisAquilis/notebooks/datasetMajors/"))
idx = np.argsort(test_pred, axis=1)[:,-5:]
idx = np.asarray(idx, dtype=int, order=None)
for i in idx[0]:
    print(i)
    print(listMajors[i] + " Accuracy: " + str(test_pred[0][i]))

1/1 [==============================] - 0s 37ms/step
105
IE Accuracy: 0.043181498
6
AE Accuracy: 0.048660643
166
SE Accuracy: 0.07701385
126
MATH Accuracy: 0.077214375
55
CS Accuracy: 0.48709688


In [45]:
subjectsCode = []
subjectsTitle = []
Dict = {}

with open("subjectCodes.txt", "r") as f:
    for line in f:
        line = line.strip("\n")
        line = line.strip("    ")
        splitLine = line.split("-")
        print(splitLine)
        splitLine[0] = splitLine[0].replace(" ", "")
        subjectsCode.append(splitLine[0])
        subjectsTitle.append(splitLine[1])
    
    
    

subjectDict = dict(zip(subjectsCode, subjectsTitle))

print(subjectDict)

['AAS ', ' Asian American Studies']
['ABE ', ' Agricultural and Biological Eng']
['ACCY ', ' Accountancy']
['ACE ', ' Agr & Consumer Economics']
['ACES ', ' Agr, Consumer, & Env Sciences']
['ADV ', ' Advertising']
['AE ', ' Aerospace Engineering']
['AFAS ', ' Air Force Aerospace Studies']
['AFRO ', ' African American Studies']
['AFST ', ' African Studies']
['AGCM ', ' Agricultural Communications']
['AGED ', ' Agricultural Education']
['AHS ', ' Applied Health Sci Courses']
['AIS ', ' American Indian Studies']
['ALEC ', ' Agricultural Leadership, Education, & Communications Program']
['ANSC ', ' Animal Sciences']
['ANTH ', ' Anthropology']
['ARAB ', ' Arabic']
['ARCH ', ' Architecture']
['ART ', ' Art']
['ARTD ', ' Art', '', 'Design']
['ARTE ', ' Art', '', 'Education']
['ARTF ', ' Art', '', 'Foundation']
['ARTH ', ' Art', '', 'History']
['ARTJ ', ' Japan House']
['ARTS ', ' Art', '', 'Studio']
['ASRM ', ' Actuarial Science & Risk Management']
['ASST ', ' Asian Studies']
['ASTR ', ' Astr

In [39]:
import json


import json
with open('subjects.json', 'w') as fp:
    json.dump(subjectDict, fp)
